In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.optim import SGD
from torch.utils.data import DataLoader

from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt

import warnings 

warnings.filterwarnings('ignore')

<br/> 

## Överblick

**Nu ska vi träna Neurala Nätverk och få lite praktiskt erfarenhet av Hyperparameters**

Vi kommer bygga och träna Neurala Nätverk för att lösa regressionsproblem.

**GRAFIKKORTSACCELERATION**

Kolla om GPU-acceleration är tillgänligt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Available device: {device}")

## Data

Vi ska använda oss av Seol bicycle demand data, och träna modeller att kunna predicta... just det, bicycle demand per dag.  

Datat kommer från https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand

**Som vanligt går ni in där för att läsa på mer om respektive feature!**

In [ ]:
demand_df = pd.read_csv('SeoulBikeData.csv', encoding='latin1')
demand_df

Nästan alla features ser rätt nice ut och värda att behålla, men vi kommer göra en förenkling. Vi tar för givet att det endast spelar roll huruvida aktuell dag är en helgdag eller inte. Dvs, vi bryr oss inte om exakt vilken dag det är. Vi kan således ta bort kolumnen 'Date' eftersom att vi har en annan kolumn 'Holiday' som anger om det är helgdag eller ej.
  

In [ ]:
demand_df = demand_df[demand_df.columns[1:]] # exludera första kolumnen (Date)
demand_df

In [ ]:
demand_df.info()

Nu återstår fyra kolumner kvar som vi behöver specialanpassa. Dels har vi Hour och Seasons, som både är kategoriska. Därefter har vi också Holiday och Functioning Day som är binära, men behöver omvandlas till siffor.

In [ ]:
print(demand_df['Holiday'].value_counts(), end = '\n\n')
print(demand_df['Functioning Day'].value_counts(), end = '\n\n')
print(demand_df['Seasons'].value_counts(), end = '\n\n')
print(demand_df['Hour'].value_counts(), end = '\n\n')

**Omvandla binära kolumner till 1/0.**

In [ ]:
binary_columns = ['Holiday', 'Functioning Day']

for column in binary_columns:
    
    first_value = demand_df[column].unique()[0] # extrahera ett av de binära värdena
    transformed_column = [1 if value == first_value else 0 for value in demand_df[column]]
    
    demand_df[column] = transformed_column

**Kolumnen Hour**

Att direkt göra en One-Hot-encoding av Hour skulle ger oss alldeles för många nya kolumner. Istället konstruerar vi färre kategorier genom att klumpa ihop
följande tidsspann

0-5

6-11

12-17

18-23



In [ ]:
new_hours = []                            # en lista som indikerar den nya kategorin för varje träningsinstans

for hour in demand_df['Hour']:

    if hour in range(0,6):
        new_hours.append(0)

    elif hour in range(6, 12):
        new_hours.append(1)
    
    elif hour in range(12, 18):
        new_hours.append(2)

    elif hour in range(18, 24):
        new_hours.append(3)

In [ ]:
demand_df['Hour'] = new_hours           # ersätt Hour med våra nya värden

demand_df['Hour'].value_counts()

**Omvandla kategoriska kolumner med One Hot Encoding**

In [ ]:
# För varje möjlig kategoriskt värde, loopa och konstruera en ny kolumn enligt ovan

categorical_columns = ['Seasons', 'Hour']

for column in categorical_columns:
    for value in set(demand_df[column].values):
    
        onehotencode = [1 if x == value else 0 for x in demand_df[column]]
        demand_df[value] = onehotencode
    

#slutligen, droppa orginalkolumnen som vi inte längre behöver    

for column in categorical_columns:
    demand_df = demand_df.drop(columns=[column])

In [ ]:
demand_df.info()

**Kontrolluppgift: Kolla så att datan är enligt förväntan, samt att alla kolumner nu är antingen int eller float**

## Dela upp data i train/test

Vi väljer återigen proportionen 90% / 10%

In [ ]:
target_column = demand_df.columns[0]
feature_columns = demand_df.columns[1:]

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(demand_df[feature_columns],         
                                                    demand_df[target_column],
                                                    test_size=0.1, 
                                                    random_state=42)

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

<br/>

**Skala data**

Nu  när vi delat upp data i train och test kan vi utföra de mer invasiva transformationerna, som att skala kolumner. **Kom dock ihåg att det är superviktigt att vi endast använder statistik från train split när vi transformerar, annars riskerar vi informationsläckage!** 

Det enda vi egentligen behöver skala om feature kolumnerna så att samtliga värden, till absolutbeloppet, inte blir särskilt mycket större än 1. Kikar vi lite snabbt på vår data så ser vi att så inte är fallet.

Vi återanvänder vårt trick för att åstadkomma detta: att helt enkelt dela respektive kolumn, med det (till absolutbeloppet) högsta värdet. Då kommer samtliga värden skalas ner, och det högsta värdet i respektive kolumn vara (i absolut värde) 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

for column in feature_columns:                               # iterera över alla input features
    
    highest_value = max(np.abs(x_train[column]))    # hitta det, till absolutbeloppet, högsta värdet i aktuella kolumn i x_train
    
    
    # dela nu aktuell kolumn i x_train med det nyfunna högsta värdet
    
    x_train[column] = x_train[column] / highest_value 
    
    # dela nu även motsvarande kolumn i x_test med SAMMA nyfunna högsta värde (från x_train)
    
    x_test[column] = x_test[column] / highest_value

**KONTROLLUPPGIFT: Kolla så att vi åstadkommit transformationerna vi eftersökte** 

<br/>

**Omvandla till Tensor**

Nu när siffrorna ser bra ut återstår det att omvandla till datatypen Tensor (optimal för PyTorch)

In [ ]:
x_train = torch.from_numpy(np.array(x_train)).type(torch.FloatTensor)
y_train = torch.from_numpy(np.array(y_train)).type(torch.FloatTensor).reshape([-1,1])

x_test = torch.from_numpy(np.array(x_test)).type(torch.FloatTensor)
y_test = torch.from_numpy(np.array(y_test)).type(torch.FloatTensor).reshape([-1,1])

**Ange för PyTorch att förbereda datan för GPU-acceleration**

In [ ]:
x_train = x_train.to(device)
y_train = y_train.to(device)

x_test = x_test.to(device)
y_test = y_test.to(device)

**Zippa ihop vår träning- och testdata**

In [ ]:
training_set = list(zip(x_train, y_train))             # lägg ihop träningsdatan så att vi direkt kan skicka in i dataloader
test_set = list(zip(x_test, y_test))                   # ditto för testdatan

<br/> 

## Skapa ett Neuralt Nätverk för regression

<br/> 

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 30)
        self.fc2 = nn.Linear(30, 30)
        self.fc3 = nn.Linear(30, 1)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
                  
        return x

 <br/>

**Skapa en instans**

In [ ]:
input_size = 18                                    # vad ska input_size vara?

model = NeuralNetwork(input_size)
model = model.to(device)                         # vi skickar över modellen till gpu - om tillgänglig

model

**Kontrollfråga: Hur måna parametrar har ditt neurala nätverk?**

<br/>

Låt oss testa vad vår initierade modell spottar ur sig för output

In [ ]:
sample = 100                                     # välj här vilken sample som helst, ange ett giltigt värde amellan 0 och 7884

sample_features = x_train[sample]                # extrahera features
sample_class = y_train[sample]                   # extrahera class (1 eller 0)

model_prediction = model(sample_features)        # predicta outcome

print('True bicycle demand               :', sample_class.item())
print('Vår models predict                :', model_prediction.item())

**Kontrollfråga** Hur tolkar du resultatet ovan?

<br/> 

## Träna

 <br/> 

In [ ]:
input_size = 18                    # vad ska input_size vara?
batch_size = 16

epochs = 20                      # default = 20
learning_rate = 0.001             # default = 0.01

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
#    initera modell, loss_function, optimizer & dataloader


#model = NeuralNetwork(input_size)
model = model.to(device)                                  # förbered modellen för GPU

optimizer = SGD(model.parameters(), lr = learning_rate)
loss_function = torch.nn.L1Loss()

train_dataloader = DataLoader(training_set,                 
                              batch_size = batch_size,       
                              shuffle=True)


# ----------------------------------------------------------------------------------------------------------------
#    träna



batch_train_losses = []

epoch_train_losses = []
epoch_evaluation_losses = []

for i in range(epochs):
    
    model.train()

    running_loss = 0
    
    for batch in train_dataloader:
        
        y_true = batch[1]
        input_features = batch[0]
        
        y_pred=model(input_features)
        loss=loss_function(y_pred, y_true)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        batch_loss = loss.item()
        batch_train_losses.append(batch_loss)
    
    epoch_average_loss = np.average(batch_train_losses[-len(train_dataloader):])
    epoch_train_losses.append(epoch_average_loss)

# ----------------------------------------------------------------------------------------------------------------
#   evalueringssektion 

    model.eval()
    
    y_true = y_test
    input_features = x_test
    
    y_pred = model(input_features)
    loss = loss_function(y_pred, y_true)
    
    evaluation_loss = loss.item()
    epoch_evaluation_losses.append(evaluation_loss)

# ----------------------------------------------------------------------------------------------------------------
#   plotta resultat 

plt.plot(epoch_train_losses, label = 'train loss')
plt.plot(epoch_evaluation_losses, label = 'test loss')
plt.legend()

plt.show()#

## Uppgifter

<br/> 

**1)**

Vi har ändrat på vad vi plottar. Lägg märke till att train loss nu är smooth! 

Vad är det vi gjort?! Är det rimtligt att göra så?

<br/>

**2)**

Titta på train- och test losskurvorna från körning ovan (med default inställningar). 

Hur tolkar du resultatet? Vad bör man göra i ett sånt här scenario?

<br/>

**3)**

Hur ser nätverket ut som du precis tränat? Rita upp den.

<br/>

**4)**

Vi ska nu utföra en random search av hyperparameters, och vi ska endast göra detta över hyperparametern antal_neurons_per_layer.

Observera följande lista

antal_neurons_per_layer = [1, 5, 10, 20, 30, 50, 100]

Gör en loop och träna nu ett nätverk med dessa värden på antal neuroner per lager.

Bygg och träna samtliga nätverk med 

antal_lager =  4

batch_size = 6

learning_rate = 0.01

epochs = 20

Vad drar du för slutsats?



**OBS, ibland kan det se ut som att nätverket inte lärt sig något alls. Och så kan det vara, vi hade otur med matematiken helt enkelt. Stora nätverk kan ibland ta mer/mindre tid att komma igång att lära sig. Testa köra igen, och hoppas på lite mer tur! Du kanske också kan testa öka antal epoker.**

**Vi kommer lära oss strategier för att motverka detta vid ett senare skede**


<br/>

<br/>